## quadtree

https://www.cs.princeton.edu/courses/archive/fall03/cs126/assignments/barnes-hut.html
http://www-inf.telecom-sudparis.eu/COURS/CSC5001/new_site/Supports/Projet/NBody/barnes_86.pdf  
https://en.wikipedia.org/wiki/Barnes%E2%80%93Hut_simulation

https://github.com/rdeits/RegionTrees.jl

https://github.com/JuliaArrays/StaticArrays.jl

In [1]:
mutable struct Cell
    corner
    size
    childrens
    mass
    center_of_mass
end;


1. If node x does not contain a body, put the new body b here.
2. If node x is an internal node, update the center-of-mass and total mass of x. Recursively insert the body b in the appropriate quadrant.
3. If node x is an external node, say containing a body named c, then there are two bodies b and c in the same region. Subdivide the region further by creating four children. Then, recursively insert both b and c into the appropriate quadrant(s). Since b and c may still end up in the same quadrant, there may be several subdivisions during a single insertion. Finally, update the center-of-mass and total mass of x. 

In [33]:
function insert!(cell, point)
    # check if node in cell?
    
    # if empty
    if cell.mass == 0
        cell.mass = 1
        cell.center_of_mass = point
    # if one point only
    elseif cell.mass == 1
        split!(cell)
        cell.mass = 2
        previous_point = cell.center_of_mass
        cell.center_of_mass = (point .+ previous_point)./2
        
        quadrant = which_quadrant(point..., cell.corner..., cell.size...)
        insert!(cell.childrens[quadrant], point)

        previousquadrant = which_quadrant(previous_point..., cell.corner..., cell.size...)
        insert!(cell.childrens[previousquadrant], previous_point)

    else
        cell.center_of_mass = (cell.center_of_mass.*cell.mass .+ 1)./(cell.mass + 1)
        cell.mass += 1
        
        quadrant = which_quadrant(point..., cell.corner..., cell.size...)
        insert!(cell.childrens[quadrant], point)
    end
            
    end;

In [34]:
function which_quadrant(x, y, cell_x, cell_y, width, height)
    quadrant = 1
    if x > cell_x + width/2
        quadrant += 1
        end;
    if y < cell_y + height/2
        quadrant += 2
        end;
    return quadrant
    end;
    

In [14]:
println( which_quadrant(.1, .1, 0, 0, 1, 1) )
println( which_quadrant(.7, .1, 0, 0, 1, 1) )
println( which_quadrant(.1, .7, 0, 0, 1, 1) )
println( which_quadrant(.8, .8, 0, 0, 1, 1) )

3
4
1
2


In [15]:
println( which_quadrant([.4 .4]..., 0, 0, 1, 1) )

3


In [24]:
function split!(cell)
    half_h = cell.size[2]/2
    half_w = cell.size[1]/2
    cell.childrens = (
        Cell(cell.corner .+ [0 half_h], cell.size./2, nothing, 0, nothing),
        Cell(cell.corner .+ [half_w half_h], cell.size./2, nothing, 0, nothing),
        Cell(cell.corner, cell.size./2, nothing, 0, nothing),
        Cell(cell.corner .+ [half_w 0], cell.size./2, nothing, 0, nothing),
    )
    end;

In [35]:
root = Cell([0, 0], [1, 1], nothing, 0, nothing)

Cell([0, 0], [1, 1], nothing, 0, nothing)

In [36]:
insert!(root, [.3 .3])

1×2 Array{Float64,2}:
 0.3  0.3

In [37]:
root

Cell([0, 0], [1, 1], nothing, 1, [0.3 0.3])

In [38]:
insert!(root, [.8 .4])

1×2 Array{Float64,2}:
 0.3  0.3

In [41]:
root.center_of_mass

1×2 Array{Float64,2}:
 0.7  0.566667

In [40]:
insert!(root, [.1 .7])

1×2 Array{Float64,2}:
 0.1  0.7